# Country wise data

I first impute country name expansions into the data set, as this will help me add income and other vairable easier. 

1. percapita income, income group, and region - done 
2. corruption - rule of law, and no-corruption from the [World Justice Project](https://worldjusticeproject.org/rule-of-law-index/)

In [1]:
# importing my cleaned transit dataset and subseting to (First) a list of all unique city and corresponding country code (Secondly) unique country codes
import pandas as pd

transit = pd.read_csv("Data/transit_cleaned.csv")
citylist = transit.drop_duplicates(subset='city')[['country', 'city']].reset_index()
citylist.drop('index', axis=1, inplace=True)
countrylist = transit.drop_duplicates(subset='country')[['country']].reset_index()
countrylist.drop('index', axis=1, inplace=True)

In [2]:
# installing pycountry to match codes with country names
!pip install pycountry
import pycountry as pyc
import re

# https://github.com/flyingcircusio/pycountry
# using the fuzzy search and matching the first result with the name. This might not be the best, we will later verify if all looks good.
countrylist['name']=countrylist['country'].apply(lambda x: pyc.countries.search_fuzzy(x)[0])
countrylist['country_name'] =  countrylist['name'].astype(str).apply(lambda x: re.search(r"name='(.*?)'", x).group(1))


In [3]:
# after inspecting, two wrong matched are found and there is exatly one city in each of these countries
# DR = Dominican Republic
# UK = United Kingdom
countrylist.set_index('country', inplace=True)
countrylist.drop('name', inplace=True, axis=1)
countrylist.loc['UK', 'country_name'] = 'United Kingdom'
countrylist.loc['DR','country_name']= 'Dominican Republic'

In [4]:
# income, region, corruption, categories
income=pd.read_csv("Data/income.csv")
region=pd.read_csv("Data/region.csv")
region.head()
income = income.set_index('Country Code').join(region.set_index('Country Code'))
income = income.reset_index()
# nan values in region are for region groups. these are not country rows. 
politics=pd.read_csv("Data/politics.csv")
politics=politics[['Country', 'Country Code','WJP Rule of Law Index: Overall Score','Factor 2: Absence of Corruption', 'Factor 6: Regulatory Enforcement']]
politics.rename(columns={'WJP Rule of Law Index: Overall Score':'rule_of_law','Factor 2: Absence of Corruption':'no_corruption', 'Factor 6: Regulatory Enforcement':'enforcement'},inplace=True)

income = income.set_index('Country Code').join(politics.set_index('Country Code'))
income = income.reset_index()
income.columns

Index(['Country Code', 'Country Name', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022',
       'Region', 'IncomeGroup', 'SpecialNotes', 'TableName', 'Unnamed: 5',
       'Country', 'rule_of_law', 'no_corruption', 'enforcement'],
      dtype='object')

In [5]:
countrylist=countrylist.reset_index()
data = pd.merge(countrylist, income[['Country Name', 'Region', 'IncomeGroup', '2021', 'rule_of_law', 'no_corruption', 'enforcement']], left_on='country_name', right_on='Country Name', how='left')

In [6]:
# assuming df is your DataFrame
df_with_nan = data[data.isnull().any(axis=1)]
df_with_nan

,country,country_name,Country Name,Region,IncomeGroup,2021,rule_of_law,no_corruption,enforcement
0,TR,Turkey,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,TW,Taiwan,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,KR,South Korea,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32,IR,"Iran, Islamic Republic of",NaN,NaN,NaN,NaN,NaN,NaN,NaN
36,IL,Israel,Israel,Middle East & North Africa,High income,52166.81216,NaN,NaN,NaN
43,CH,Switzerland,Switzerland,Europe & Central Asia,High income,91991.60046,NaN,NaN,NaN
49,SA,Saudi Arabia,Saudi Arabia,Middle East & North Africa,High income,24160.67605,NaN,NaN,NaN
50,QA,Qatar,Qatar,Middle East & North Africa,High income,66838.32764,NaN,NaN,NaN
51,BH,Bahrain,Bahrain,Middle East & North Africa,High income,26860.07250,NaN,NaN,NaN
53,EG,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,NaN


United States
Russian Federation
Czechia
United Kingdom
Turkiye
Korea, Rep.
Iran, Islamic Rep.

In [7]:
# replacing names that are in the income dataset into the main. Taiwan is missing
countrylist=countrylist.set_index('country')
countrylist.at['HK','country_name']= 'Hong Kong SAR, China'
countrylist.at['EG','country_name']= 'Egypt, Arab Rep.'
countrylist.at['TR','country_name']= 'Turkiye'
countrylist.at['KR','country_name']= 'Korea, Rep.'
countrylist.at['IR','country_name']= 'Iran, Islamic Rep.'

In [8]:
# assuming df is your DataFrame
countrylist=countrylist.reset_index()
countrylist = pd.merge(countrylist, income[['Country Name', 'Region', 'IncomeGroup', '2021', 'rule_of_law', 'no_corruption', 'enforcement']], left_on='country_name', right_on='Country Name', how='left')
countrylist.at[16,'country_name']= 'Taiwan'
# data from IMF: https://www.imf.org/external/datamapper/NGDPDPC@WEO/ADVEC/WEOWORLD/TWN/CHN
countrylist.at[16,'2021']= 33190
countrylist.drop('Country Name', axis=1, inplace=True)
countrylist.loc[16]

country               TW
country_name      Taiwan
Region               NaN
IncomeGroup          NaN
2021             33190.0
rule_of_law          NaN
no_corruption        NaN
enforcement          NaN
Name: 16, dtype: object

In [9]:
countrylist.at[16, 'Region']= 'East Asia & Pacific'
countrylist.at[16, 'IncomeGroup']= 'High income'
countrylist.loc[16]

country                           TW
country_name                  Taiwan
Region           East Asia & Pacific
IncomeGroup              High income
2021                         33190.0
rule_of_law                      NaN
no_corruption                    NaN
enforcement                      NaN
Name: 16, dtype: object

In [10]:
# Do at the end. 
# merging it back to the city list for a check
citylist=citylist.set_index('country').join(countrylist.set_index('country'))

In [11]:
citylist.sample(10)

,city,country_name,Region,IncomeGroup,2021,rule_of_law,no_corruption,enforcement
country,,,,,,,,
KR,Incheon,"Korea, Rep.",East Asia & Pacific,High income,34997.781640,0.74,0.67,0.74
CA,Montreal,Canada,North America,High income,52358.621640,0.80,0.82,0.80
RU,Nizhniy Novgorod,Russian Federation,Europe & Central Asia,Upper middle income,12593.157230,0.46,0.42,0.48
IN,Surat,India,South Asia,Lower middle income,2238.127139,0.50,0.40,0.48
US,Chicago,United States,North America,High income,70219.472450,0.69,0.72,0.71
TW,Taipei,Taiwan,East Asia & Pacific,High income,33190.000000,NaN,NaN,NaN
TR,Istanbul,Turkiye,Europe & Central Asia,Upper middle income,9661.227734,0.42,0.46,0.41
FR,Toulouse,France,Europe & Central Asia,High income,43658.978980,0.72,0.74,0.76
CN,Kunming,China,East Asia & Pacific,Upper middle income,12617.504990,0.47,0.53,0.49


In [12]:
citylist.reset_index()

,country,city,country_name,Region,IncomeGroup,2021,rule_of_law,no_corruption,enforcement
0,AE,Dubai,United Arab Emirates,Middle East & North Africa,High income,44315.554180,0.64,0.80,0.71
1,AR,Buenos Aires,Argentina,Latin America & Caribbean,Upper middle income,10636.115530,0.56,0.48,0.50
2,AT,Vienna,Austria,Europe & Central Asia,High income,53637.705710,0.81,0.82,0.82
3,AU,Sydney,Australia,East Asia & Pacific,High income,60444.502360,0.79,0.81,0.81
4,AU,Melbourne,Australia,East Asia & Pacific,High income,60444.502360,0.79,0.81,0.81
...,...,...,...,...,...,...,...,...,...
179,US,Chicago,United States,North America,High income,70219.472450,0.69,0.72,0.71
180,US,Philadelphia,United States,North America,High income,70219.472450,0.69,0.72,0.71
181,UZ,Tashkent,Uzbekistan,Europe & Central Asia,Lower middle income,1993.424478,0.49,0.46,0.44
182,VN,Hanoi,Vietnam,East Asia & Pacific,Lower middle income,3756.489122,0.49,0.41,0.44


In [13]:
transit=transit[['country', 'city', 'phase', 'start_year', 'end_year',
       'rr', 'length', 'tunnelper', 'tunnel', 'elevated', 'atgrade',
       'stations', 'platform_length_meters', 'anglo', 'real_cost_2021',
       'cost_km_2021']]

In [14]:
citylist.columns

Index(['city', 'country_name', 'Region', 'IncomeGroup', '2021', 'rule_of_law',
       'no_corruption', 'enforcement'],
      dtype='object')

In [15]:
transit_add = pd.merge(transit, citylist, left_on='city',  right_on='city', how='left')


In [16]:
transit_add.to_csv("Data/transit_income.csv")

In [18]:
transit_add.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 835 entries, 0 to 834
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country                 835 non-null    object 
 1   city                    835 non-null    object 
 2   phase                   835 non-null    object 
 3   start_year              835 non-null    int64  
 4   end_year                835 non-null    int64  
 5   rr                      835 non-null    float64
 6   length                  835 non-null    float64
 7   tunnelper               835 non-null    float64
 8   tunnel                  835 non-null    float64
 9   elevated                483 non-null    float64
 10  atgrade                 481 non-null    float64
 11  stations                835 non-null    float64
 12  platform_length_meters  406 non-null    object 
 13  anglo                   835 non-null    float64
 14  real_cost_2021          835 non-null    fl